## Baseline Submission: Toxic Language Classification 
**w207 Spring 2018 - Final Project Baseline**

**Team: Paul, Walt, Yisang, Joe**



### Project Description 

Our challenge is to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate.  The toxic language data set is sourced from Wikipedia and available as a public kaggle data set. 

Our goal is to use various machine learning techniques used in class to develop high quality ML models and pipelines.  

1. Exercise and build upon concepts covered in class and test out at least 3 kinds of supervised models:
    a. Regression (LASSO, Logistic)
    b. Trees (RF, XGBoost)
    c. DeepLearning (Tensorflow)
2. Using stacking/ensembling methods for improving prediction metrics (K-Means, anomaly detection)
3. Using unsupervised methods for feature engineering/selection

For the baseline proposal, this file contains a first pass run through from data preprocessing to model evaluation using a regression model pipeline. 

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge




### Data Ingestion

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import time

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import auc

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 




#NLTK imports
import string

import nltk
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk.tokenize import punkt as punkt
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

# These imports enable the use of NLTKPreprocessor in an sklearn Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


#scipy imports
from scipy.sparse import hstack

#General imports
import pprint

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/Users/burgew/Library/Python/2.7/lib/python/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/burgew/Library/Python/2.7/lib/python/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# read frames localy through csv
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

np.random.seed(455)

# Random index generator for splitting training data
# Note: Each rerun of cell will create new splits.
randIndexCut = np.random.rand(len(train_df)) < 0.7

#S plit up data
test_data = test_df["comment_text"]
dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]

print 'total training observations:', train_df.shape[0]
print 'training data shape:', train_data.shape
print 'training label shape:', train_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', target_names

total training observations: 159571
training data shape: (111691,)
training label shape: (111691, 6)
dev label shape: (47880, 6)
labels names: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


### Text Processing

In [5]:
nltk.download('stopwords')

class NLTKPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None,
                 lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = stopwords or set(sw.words('english'))
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def inverse_transform(self, X):
        return [" ".join(doc) for doc in X]

    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):

        # Break the document into sentences
        for sent in sent_tokenize(unicode(document,'utf-8')):

            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token

                # If stopword, ignore token and continue
                if token in self.stopwords:
                    continue

                # If punctuation, ignore token and continue
                if all(char in self.punct for char in token):
                    continue

                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/burgew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Starting preprocessing of training data...
Completed tokenization/preprocessing of training data in 588.66 seconds
Starting vectorization of training data...
Completed vectorization of training data in 3.33 seconds
Starting preprocessing of dev data...
Completed tokenization/preprocessing of dev data in 242.46 seconds

Starting vectorization of dev data...
Completed vectorization of dev data in 1.12 seconds

Vocabulary (tfidf) size is: 15000
Sample vocabulary from TfidfVectorizer:
         count
""—          0
(“           1
(→           2
(☎)          3
(𒁳)         4
(｡◕‿◕｡)      5
)‎           6
,”           7
-•           8
.·           9
None
...
     count
➨    14990
・    14991
水    14992
竜龙   14993
聖やや  14994
見学   14995
迷惑   14996
連絡   14997
雲    14998
，    14999
None
Number of nonzero entries in matrix: 2855615


,counts
42,4
43,3
44,1
51,2
56,3
59,1
65,3
79,2
86,2
151,1


### Final Text Preprocessing - training data

In [22]:
pp = pprint.PrettyPrinter(indent=4)

np.random.seed(455)

# This preprocessor will be used to process data prior to vectorization
nltkPreprocessor = NLTKPreprocessor()
    
# Note that this vectorizer is created with a passthru tokenizer(identity), no preprocessor and no lowercasing
# This is to account for the NLTKPreprocessor already taking care of these.
tfidfVector = TfidfVectorizer(ngram_range=(1,1), min_df=5, max_features=15000,
                              tokenizer=identity, preprocessor=None, lowercase=False)

print "Starting final preprocessing of training data..."
start_train_preproc = time.time()
trainPreprocData = nltkPreprocessor.fit_transform(train_df["comment_text"])
finish_train_preproc = time.time()
print "Completed tokenization/preprocessing of training data in {:.2f} seconds".format((finish_train_preproc-start_train_preproc))

print "Starting final preprocessing of test data..."
start_test_preproc = time.time()
testPreprocData = nltkPreprocessor.transform(test_df["comment_text"])
finish_test_preproc = time.time()
print "Completed tokenization/preprocessing of test data in {:.2f} seconds".format((finish_test_preproc-start_test_preproc))

print "Starting vectorization of training data..."
start_train_vectors = time.time()
finalTrainCounts = tfidfVector.fit_transform(trainPreprocData)
finish_train_vectors = time.time()
print "Completed vectorization of training data in {:.2f} seconds".format((finish_train_vectors-start_train_vectors))

print "Starting vectorization of test data..."
start_test_vectors = time.time()
finalTestCounts = tfidfVector.transform(testPreprocData)
finish_test_vectors = time.time()
print "Completed vectorization of test data in {:.2f} seconds".format((finish_test_vectors-start_test_vectors))

Starting final preprocessing of training data...
Completed tokenization/preprocessing of training data in 857.89 seconds
Starting final preprocessing of test data...
Completed tokenization/preprocessing of test data in 771.28 seconds
Starting vectorization of training data...
Completed vectorization of training data in 5.24 seconds
Starting vectorization of test data...
Completed vectorization of test data in 3.75 seconds


### Final LSA Feature Selection - training data

In [23]:
target_components = len(tfidfVector.vocabulary_)/10
svd = TruncatedSVD(n_components=target_components, algorithm='arpack')
print "Starting LSA on train counts with {} components...".format(target_components)
train_start=time.time()
lsaTrainCounts = svd.fit_transform(finalTraincounts)
train_stop=time.time()
print "Train counts transform took {:.2f} seconds.".format(train_stop-train_start)

target_components = len(tfidfVector.vocabulary_)/10
svd = TruncatedSVD(n_components=target_components, algorithm='arpack')
print "Starting LSA on test counts with {} components...".format(target_components)
train_start=time.time()
lsaTestCounts = svd.fit_transform(finalTestCounts)
train_stop=time.time()
print "Test counts transform took {:.2f} seconds.".format(train_stop-train_start)

Starting LSA on train counts with 1500 components...
Train counts transform took 351.07 seconds.
Starting LSA on test counts with 1500 components...
Test counts transform took 345.80 seconds.


### Final MLPClassifier Training and Submission

In [24]:
prediction_submission = pd.DataFrame()
prediction_submission["id"] = test_df["id"]

print("Training with MLPClassifier (shallow/wide Net)")
classifier = MLPClassifier(hidden_layer_sizes=(12,6), activation='tanh', learning_rate='adaptive')

# Training with shallow/wide Neural Net 
for name in target_names:
    
    label_train_start = time.time()
    classifier = MLPClassifier(hidden_layer_sizes=(12,6), activation='tanh', learning_rate='adaptive')
    classifier.fit(lsaTrainCounts, train_df[name])
    label_train_finish = time.time()
    print('Training for class {} completed, after {:.2f} minutes.'.format(name, 
                                                                          (label_train_finish-label_train_start)/60))
    label_predict_start = time.time()
    prediction_submission[name] = classifier.predict_proba(lsaTestCounts)[:, 1]
    label_predict_finish = time.time()
    print('Prediction for class {} completed, after {:.2f} minutes.'.format(name,
                                                                    (label_predict_finish-label_predict_start)/60))


    

    
print(prediction_submission.head(10)) # print frame output 
prediction_submission.to_csv("submission.csv")

Training with MLPClassifier (shallow/wide Net)
Training for class toxic completed, after 8.02 minutes.
Prediction for class toxic completed, after 0.13 minutes.
Training for class severe_toxic completed, after 3.23 minutes.
Prediction for class severe_toxic completed, after 0.01 minutes.
Training for class obscene completed, after 4.65 minutes.
Prediction for class obscene completed, after 0.01 minutes.
Training for class threat completed, after 1.46 minutes.
Prediction for class threat completed, after 0.01 minutes.
Training for class insult completed, after 5.89 minutes.
Prediction for class insult completed, after 0.08 minutes.
Training for class identity_hate completed, after 3.03 minutes.
Prediction for class identity_hate completed, after 0.01 minutes.
                 id         toxic  severe_toxic       obscene    threat  \
0  00001cee341fdb12  9.413830e-04  3.019389e-06  8.532108e-01  0.000011   
1  0000247867823ef7  6.041182e-06  1.038509e-06  2.992874e-07  0.000010   
2  000

### Submission - based on test preprocessing, LSA feature selection and MLPClassifier training

In [ ]:
prediction_submission = pd.DataFrame()
prediction_submission["id"] = test_df["id"]

# new vector object for all train data for submission
finalTrainVector = CountVectorizer()
finalTrainCount = finalTrainVector.fit_transform(train_df["comment_text"])

# TODO: Using pipelines can clean up repetitive processes
# test set up
#testVector = CountVectorizer()
testCount = finalTrainVector.transform(test_df["comment_text"])

for name in target_names:
    classifier = LogisticRegression(solver='sag') #sag is one kind of solver optimize for multi-label
    clf = classifier.fit(finalTrainCount, train_df[name])
    prediction_submission[name] = clf.predict_proba(testCount)[:, 1]
    #print(prediction_submission)

    
print(prediction_submission.head(10)) # print frame output 
prediction_submission.to_csv("submission.csv")

### Submission

In [ ]:
prediction_submission = pd.DataFrame()
prediction_submission["id"] = test_df["id"]

# new vector object for all train data for submission
finalTrainVector = CountVectorizer()
finalTrainCount = finalTrainVector.fit_transform(train_df["comment_text"])

# TODO: Using pipelines can clean up repetitive processes
# test set up
#testVector = CountVectorizer()
testCount = finalTrainVector.transform(test_df["comment_text"])

for name in target_names:
    classifier = LogisticRegression(solver='sag') #sag is one kind of solver optimize for multi-label
    clf = classifier.fit(finalTrainCount, train_df[name])
    prediction_submission[name] = clf.predict_proba(testCount)[:, 1]
    #print(prediction_submission)

    
print(prediction_submission.head(10)) # print frame output 
prediction_submission.to_csv("submission.csv")

The frame contains the output for each class and is saved in a pandas data frame.  